In [37]:
import fastf1
import pandas as pd
import time

In [2]:
# Get historical data. Doesn't include 2025.
year = 2022
meetings_df = pd.DataFrame()
while year<2025:
    meeting = fastf1.get_event_schedule(year)
    meeting = meeting.loc[:,["RoundNumber","Country","Location","EventFormat"]]
    meeting['Year'] = year
    meetings_df = pd.concat([meetings_df,meeting],axis=0)
    year += 1

condition = meetings_df["EventFormat"] == "testing"
meetings_df = meetings_df[~condition]


req         WARNING 	DEFAULT CACHE ENABLED! (814.73 MB) C:\Users\rohan\AppData\Local\Temp\fastf1


In [18]:
meetings_df

,RoundNumber,Country,Location,EventFormat,Year
2,1,Bahrain,Sakhir,conventional,2022
3,2,Saudi Arabia,Jeddah,conventional,2022
4,3,Australia,Melbourne,conventional,2022
5,4,Italy,Imola,sprint,2022
6,5,United States,Miami,conventional,2022
...,...,...,...,...,...
20,20,Mexico,Mexico City,conventional,2024
21,21,Brazil,São Paulo,sprint_qualifying,2024
22,22,United States,Las Vegas,conventional,2024
23,23,Qatar,Lusail,sprint_qualifying,2024


In [130]:
event_sprint_2022 = fastf1.get_event(2022,4)
event_sprint_2022

RoundNumber                                                          4
Country                                                          Italy
Location                                                         Imola
OfficialEventName    FORMULA 1 ROLEX GRAN PREMIO DEL MADE IN ITALY ...
EventDate                                          2022-04-24 00:00:00
EventName                                    Emilia Romagna Grand Prix
EventFormat                                                     sprint
Session1                                                    Practice 1
Session1Date                                 2022-04-22 13:30:00+02:00
Session1DateUtc                                    2022-04-22 11:30:00
Session2                                                    Qualifying
Session2Date                                 2022-04-22 17:00:00+02:00
Session2DateUtc                                    2022-04-22 15:00:00
Session3                                                    Practice 2
Sessio

In [131]:
event_sprint_2023 = fastf1.get_event(2023,4)
event_sprint_2023

RoundNumber                                             4
Country                                        Azerbaijan
Location                                             Baku
OfficialEventName    FORMULA 1 AZERBAIJAN GRAND PRIX 2023
EventDate                             2023-04-30 00:00:00
EventName                           Azerbaijan Grand Prix
EventFormat                               sprint_shootout
Session1                                       Practice 1
Session1Date                    2023-04-28 13:30:00+04:00
Session1DateUtc                       2023-04-28 09:30:00
Session2                                       Qualifying
Session2Date                    2023-04-28 17:00:00+04:00
Session2DateUtc                       2023-04-28 13:00:00
Session3                                  Sprint Shootout
Session3Date                    2023-04-29 12:30:00+04:00
Session3DateUtc                       2023-04-29 08:30:00
Session4                                           Sprint
Session4Date  

In [5]:
event_sprint_2024 = fastf1.get_event(2024,6)
event_sprint_2024

RoundNumber                                                   6
Country                                           United States
Location                                                  Miami
OfficialEventName    FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2024
EventDate                                   2024-05-05 00:00:00
EventName                                      Miami Grand Prix
EventFormat                                   sprint_qualifying
Session1                                             Practice 1
Session1Date                          2024-05-03 12:30:00-04:00
Session1DateUtc                             2024-05-03 16:30:00
Session2                                      Sprint Qualifying
Session2Date                          2024-05-03 16:30:00-04:00
Session2DateUtc                             2024-05-03 20:30:00
Session3                                                 Sprint
Session3Date                          2024-05-04 12:00:00-04:00
Session3DateUtc                         

In [27]:
#how to get session from event 
qual = fastf1.get_session(2024,6,'Q')
qual.load()

race = fastf1.get_session(2024,6,'R')
race.load()

sprint = fastf1.get_session(2024,6,'S')
sprint.load()

core           INFO 	Loading data for Miami Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '55', '11', '4', '81', '63', '44', '27', '22', '18', '10', '31', '23', '14', '77', '2', '3', '20', '24']
core           INFO 	Loading data for Miami Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data f

In [209]:
#For EventFormat == sprint_shootout or EventFormat == sprint_qualifying

# For Qualifying 
# data from session.results
qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

#For Race 
#data from session.results
race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

# data from session.laps
race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
    'Sector1Time': 'mean',
    'Sector2Time': 'mean',
    'Sector3Time': 'mean',
    'SpeedST': 'mean',
    'Stint': 'max'
}).reset_index()

# outer join dfs --> complete data for a single race session 
merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

# Merge qual and race data 
merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

# Get Sprint data
sprint_results_df = sprint.results.loc[:,["DriverNumber","GridPosition","Position"]]
sprint_results_df = sprint_results_df.rename(columns={"GridPosition":"Sprint_Qual_Position","Position":"Sprint_Race_Position"})

# Merge Qual + Sprint + Race data
merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

merged_qual_race_df["Event_Type"] = "Sprint"

In [175]:
# For EventFormat == sprint

## note its "S" for sprint

# For Qualifying 
# data from session.results
qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

# For Race
#data from session.results
race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

# data from session.laps
race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
    'Sector1Time': 'mean',
    'Sector2Time': 'mean',
    'Sector3Time': 'mean',
    'SpeedST': 'mean',
    'Stint': 'max'
}).reset_index()

# outer join dfs --> complete data for a single race session 
merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

# Merge qual and race data 
merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

# Get Sprint data
sprint_results_df = sprint.results.loc[:,["DriverNumber","Position"]]
sprint_results_df = sprint_results_df.rename(columns={"Position":"Sprint_Race_Position"})

# Merge Qual + Sprint + Race data
merged_qual_race_df["Sprint_Qual_Position"] = merged_qual_race_df["Qual_Position"]
merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

merged_qual_race_df["Event_Type"] = "Sprint"


In [35]:
#For EventFormat == conventional

# For Qualifying 
# data from session.results
qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

# For Race
#data from session.results
race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

# data from session.laps
race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","LapTime","SpeedST","Stint"]]
for sector in ['LapTime','Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
    'Sector1Time': 'mean',
    'Sector2Time': 'mean',
    'Sector3Time': 'mean',
    'LapTime': 'mean',
    'SpeedST': 'mean',
    'Stint': 'max'
}).reset_index()

# outer join dfs --> complete data for a single race session 
merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

# Merge qual and race data 
merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

merged_qual_race_df["Sprint_Qual_Position"] = None
merged_qual_race_df["Sprint_Race_Position"] = None
merged_qual_race_df["Event_Type"] = "Race"

In [20]:
def get_event_info(year,round_num,event_format):
    
    qual = fastf1.get_session(year,round_num,'Q')
    qual.load()
    race = fastf1.get_session(year,round_num,'R')
    race.load()

    if event_format != "conventional":
        sprint = fastf1.get_session(year,round_num,'S')
        sprint.load()

    # For Qualifying 
    # data from session.results
    qual_results_df = qual.results.loc[:,["DriverNumber","Position"]]
    qual_results_df = qual_results_df.rename(columns={"Position":"Qual_Position"})

    # For Race
    # data from session.results
    race_results_df = race.results.loc[:,["DriverNumber","BroadcastName","TeamName","Position"]]

    # data from session.laps
    race_laps_df = race.laps.loc[:,["DriverNumber","Sector1Time","Sector2Time","Sector3Time","SpeedST","Stint"]]
    for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']: race_laps_df[sector] = race_laps_df[sector] / pd.Timedelta(seconds=1)
    final_race_laps_df = race_laps_df.groupby('DriverNumber').agg({
        'Sector1Time': 'mean',
        'Sector2Time': 'mean',
        'Sector3Time': 'mean',
        'SpeedST': 'mean',
        'Stint': 'max'
    }).reset_index()

    # outer join dfs --> complete data for a single race session 
    merged_race_df = pd.merge(race_results_df,final_race_laps_df,on="DriverNumber",how="outer")

    # Merge qual and race data 
    merged_qual_race_df = pd.merge(qual_results_df,merged_race_df,on="DriverNumber",how="outer")
    merged_qual_race_df = merged_qual_race_df.rename(columns={"Position":"Race_Position"})

    if event_format == "conventional":
        merged_qual_race_df["Sprint_Qual_Position"] = None
        merged_qual_race_df["Sprint_Race_Position"] = None
        merged_qual_race_df["Event_Type"] = "Race"

    elif event_format == "sprint":
        # Get Sprint data
        sprint_results_df = sprint.results.loc[:,["DriverNumber","Position"]]
        sprint_results_df = sprint_results_df.rename(columns={"Position":"Sprint_Race_Position"})

        # Merge Qual + Sprint + Race data
        merged_qual_race_df["Sprint_Qual_Position"] = merged_qual_race_df["Qual_Position"]
        merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

        merged_qual_race_df["Event_Type"] = "Sprint"

    elif event_format == "sprint_shootout" or event_format == "sprint_qualifying":
        # Get Sprint data
        sprint_results_df = sprint.results.loc[:,["DriverNumber","GridPosition","Position"]]
        sprint_results_df = sprint_results_df.rename(columns={"GridPosition":"Sprint_Qual_Position","Position":"Sprint_Race_Position"})

        # Merge Qual + Sprint + Race data
        merged_qual_race_df = pd.merge(merged_qual_race_df,sprint_results_df,on="DriverNumber",how="outer")

        merged_qual_race_df["Event_Type"] = "Sprint"

    else:
        raise Exception(f"Unknown event format: {event_format}")
    
    return merged_qual_race_df

In [38]:
def get_data():  
    final_df = pd.DataFrame()
    for index,row in meetings_df.iterrows():
        round_num = row["RoundNumber"]
        country = row["Country"]
        location = row["Location"]
        EventFormat = row["EventFormat"]
        year = row["Year"]

        print(f"{index}: {round_num}, {country}, {location}, {EventFormat}, {year}")
        time.sleep(5)
        
        meeting_data_df = get_event_info(year,round_num,EventFormat)

        meeting_data_df["Round_Number"] = round_num
        meeting_data_df["Country"] = country
        meeting_data_df["Location"] = location
        meeting_data_df["Year"] = year

        # reorganize columns
        new_col_order = ["DriverNumber","BroadcastName","TeamName","Round_Number","Country","Location","Year","Event_Type",
                        "Sprint_Qual_Position","Sprint_Race_Position","Qual_Position","Sector1Time","Sector2Time","Sector3Time",
                        "SpeedST","Stint","Race_Position"]
        meeting_data_df = meeting_data_df[new_col_order]

        final_df = pd.concat([final_df,meeting_data_df],axis=0)

    return final_df

In [21]:
d = get_data()
d.to_csv("data.csv")

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '55', '11', '44', '77', '20', '14', '63', '10', '31', '47', '4', '23', '24', '22', '27', '3', '18', '6']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count

In [22]:
d

,DriverNumber,BroadcastName,TeamName,Round_Number,Country,Location,Year,Event_Type,Sprint_Qual_Position,Sprint_Race_Position,Qual_Position,Sector1Time,Sector2Time,Sector3Time,SpeedST,Stint,Race_Position
0,1,M VERSTAPPEN,Red Bull Racing,1,Bahrain,Sakhir,2022,Race,None,None,2.0,33.720943,44.042333,25.901444,278.895833,4.0,19.0
1,10,P GASLY,AlphaTauri,1,Bahrain,Sakhir,2022,Race,None,None,10.0,32.808465,43.315864,24.596659,291.687500,3.0,20.0
2,11,S PEREZ,Red Bull Racing,1,Bahrain,Sakhir,2022,Race,None,None,4.0,33.359873,44.062893,25.503964,282.716981,4.0,18.0
3,14,F ALONSO,Alpine,1,Bahrain,Sakhir,2022,Race,None,None,8.0,33.591357,44.109825,25.341860,284.638298,4.0,9.0
4,16,C LECLERC,Ferrari,1,Bahrain,Sakhir,2022,Race,None,None,1.0,33.484018,43.873719,25.345807,277.478261,4.0,1.0
5,18,L STROLL,Aston Martin,1,Bahrain,Sakhir,2022,Race,None,None,19.0,33.680732,44.398860,25.342825,278.592593,4.0,12.0
6,20,K MAGNUSSEN,Haas F1 Team,1,Bahrain,Sakhir,2022,Race,None,None,7.0,33.276446,44.217982,25.435175,287.777778,4.0,5.0
7,22,Y TSUNODA,AlphaTauri,1,Bahrain,Sakhir,2022,Race,None,None,16.0,33.432500,44.152632,25.398246,290.641509,4.0,8.0
8,23,A ALBON,Williams,1,Bahrain,Sakhir,2022,Race,None,None,14.0,33.609643,44.659895,25.315807,290.659574,4.0,13.0
9,24,G ZHOU,Alfa Romeo,1,Bahrain,Sakhir,2022,Race,None,None,15.0,33.394607,44.246474,25.358895,289.576923,4.0,10.0
